In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import requests
from tqdm import tqdm
import pickle

KEGGPrefix = 'ftu'

url = "http://rest.kegg.jp/link/pathway/" + KEGGPrefix

paePathways = requests.get(url).text
pathDict = {}
for line in paePathways.strip().split("\n"):
    try:
        loctag, path = line.strip().split("\t")  # Fixed: correct order
        loctag = loctag.replace(KEGGPrefix + ':', "")
        path = path.replace("path:", "")
        if path not in pathDict.keys():
            pathDict[path] = []
        pathDict[path].append(loctag)
    except ValueError:
        continue

gmtFileName = KEGGPrefix + "_kegg_pathways.gmt"

with open(gmtFileName, "w") as gmt:
    for pw in pathDict:
        description = f"http://www.kegg.jp/dbget-bin/www_bget?{pw}"
        genes = pathDict[pw]
        gmt.write(f"{pw}\t{description}\t" + "\t".join(genes) + "\n")

pathDef = {}

for i in tqdm(pathDict.keys()):
    url = f"http://rest.kegg.jp/get/{i}"
    res = requests.get(url)
    for line in res.text.splitlines():
        if line.startswith("NAME"):
            pathDef[i] = line.split(sep = " - ")[0].replace("NAME", "").strip()
            break

pickleUrl = KEGGPrefix + '_descriptions.pkl'

with open(pickleUrl, 'wb') as f:
    pickle.dump(pathDef, f)

In [ ]:
# Check the GMT file
with open(gmtFileName, "r") as gmt:
    for _ in range(5):  # Print first 5 lines
        print(gmt.readline().strip())